<a href="https://colab.research.google.com/github/aokozlova/esc_data/blob/main/ESC_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import re
import random

In [2]:
from google.colab import auth
auth.authenticate_user()



In [3]:
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe
creds, _ = default()

gc = gspread.authorize(creds)
default_spreadsheet_title = 'ESC_data'

#Getting winners and links for the events
I am collecting data since 2000 up to 2022

In [4]:
def collect_table(r, table_selector, number_of_links, collect_until=None):
  soup = BeautifulSoup(r.text,'html.parser')
  table = soup.select_one(table_selector)
  columns = [th.text for th in table.find_all('th')]
  columns = columns +  ['link' + str(i) for i in range(0, number_of_links)]

  data = []
  rows = table.tbody.find_all('tr')
  for row in rows:
    cells = row.find_all('td')
    cells = [c.text.strip() for c in cells] + [c.find('a')['href'] for c in cells[:collect_until] if c.find('a')]
    data.append([c for c in cells])
  df = pd.DataFrame(data = data, columns = columns)
  return df

In [5]:
def add_data(data, worksheet_title, spreadsheet=default_spreadsheet_title):
    shs = gc.openall()
    sprsheet_exists = any(s.title == spreadsheet for s in shs)
    if sprsheet_exists:
        spreadsheet = gc.open(spreadsheet)
    else:
        spreadsheet = gc.create(spreadsheet)

    wshs = spreadsheet.worksheets()
    worksheet_exists = any(w.title == worksheet_title for w in wshs)

    if worksheet_exists:
        worksheet = spreadsheet.worksheet(worksheet_title)
        spreadsheet.del_worksheet(worksheet)

    if isinstance(data, pd.DataFrame):
        worksheet = spreadsheet.add_worksheet(title=worksheet_title, rows=len(data), cols=len(data.columns))
        set_with_dataframe(worksheet, data)
    elif isinstance(data, list):
        worksheet = spreadsheet.add_worksheet(title=worksheet_title, rows=len(data), cols=1)
        for i, value in enumerate(data):
            row = i + 1
            worksheet.update(f'A{row}', value)

In [ ]:
esc_base_url = "https://eurovision.tv/history"
all_data = []
number_of_links = 5

for i in range(0,2):
  params = {'search': '', 'page': i}
  r = requests.get(url=esc_base_url, params=params, headers={'User-Agent': 'Mozilla/5.0'})
  
  if r.ok:
    df = collect_table(r, '.cols-7',number_of_links)
    all_data.append(df)
  else:
    print(r.status_code)


4 song titles are missing. I'll collect them later

In [ ]:
winners = pd.concat(all_data, ignore_index=True)
winners = winners[['Year','City','Winners','Participant','Song','Points','link0']].rename(columns = {'link0':'event_link'})
winners = winners[winners['Year'] >='2000']
winners['event_link'] = 'https://eurovision.tv' + winners['event_link']
winners

,Year,City,Winners,Participant,Song,Points,event_link
0,2022,Turin,Ukraine,Kalush Orchestra,Stefania,631,https://eurovision.tv//event/turin-2022
1,2021,Rotterdam,Italy,Måneskin,Zitti E Buoni,524,https://eurovision.tv//event/rotterdam-2021
2,2019,Tel Aviv,Netherlands,Duncan Laurence,Arcade,498,https://eurovision.tv//event/tel-aviv-2019
3,2018,Lisbon,Israel,Netta,TOY,529,https://eurovision.tv//event/lisbon-2018
4,2017,Kyiv,Portugal,Salvador Sobral,Amar Pelos Dois,758,https://eurovision.tv//event/kyiv-2017
5,2016,Stockholm,Ukraine,Jamala,1944,534,https://eurovision.tv//event/stockholm-2016
6,2015,Vienna,Sweden,Måns Zelmerlöw,Heroes,365,https://eurovision.tv//event/vienna-2015
7,2014,Copenhagen,Austria,Conchita Wurst,Rise Like a Phoenix,290,https://eurovision.tv//event/copenhagen-2014
8,2013,Malmö,Denmark,Emmelie de Forest,Only Teardrops,281,https://eurovision.tv//event/malmo-2013
9,2012,Baku,Sweden,Loreen,Euphoria,372,https://eurovision.tv//event/baku-2012


In [ ]:
add_data(winners, 'winners', default_spreadsheet_title)

#Collecting list of events

A Semi-Final was introduced in 2004. Growing interest lead to the introduction of a second Semi-Final in 2008.

##Generating links for shows

In [ ]:
spreadsheet = gc.open(default_spreadsheet_title)
worksheet = spreadsheet.worksheet('winners')
event_links = worksheet.col_values(7)[1:]

def sf(i):
  if i < 2004: return ['/final']
  if i < 2008: return ['/semi-final','/grand-final']
  return ['/first-semi-final', '/second-semi-final', '/grand-final']

def get_shows_links(url):
  number_of_events = {str(i): sf(i) for i in range(2000,2023)}
  links = []
  year = re.search(r'[0-9]{4}$', url).group()
  for e in number_of_events[year]:
    links.append(url + e)
  return links

shows_links = []
for l in event_links:
  shows_links.extend(get_shows_links(l))

add_data(shows_links, 'shows_links')


##Collecting tables with participants of every show

In [ ]:
worksheet = gc.open(default_spreadsheet_title).worksheet('shows_links')
shows_links = worksheet.col_values(1)
shows_links[:10]

['https://eurovision.tv/event/turin-2022/first-semi-final',
 'https://eurovision.tv/event/turin-2022/second-semi-final',
 'https://eurovision.tv/event/turin-2022/grand-final',
 'https://eurovision.tv/event/rotterdam-2021/first-semi-final',
 'https://eurovision.tv/event/rotterdam-2021/second-semi-final',
 'https://eurovision.tv/event/rotterdam-2021/grand-final',
 'https://eurovision.tv/event/tel-aviv-2019/first-semi-final',
 'https://eurovision.tv/event/tel-aviv-2019/second-semi-final',
 'https://eurovision.tv/event/tel-aviv-2019/grand-final',
 'https://eurovision.tv/event/lisbon-2018/first-semi-final']

In [ ]:
all_data = []
number_of_links = 2
collect_until = 4

for url in shows_links:
  r = requests.get(url=url, headers={'User-Agent': 'Mozilla/5.0'})
  if r.ok:
    df = collect_table(r, '.cols-7',number_of_links, collect_until)
    df['event'] = re.search(r'/([a-z-0-9]+)/[a-z-0-9]+$', url).group(1)
    df['show'] = re.search(r'[a-z-]+$', url).group()
    all_data.append(df)
    print(url)
  else:
    print(r.status_code)
  sleep(random.randrange(3,7,1))
shows = pd.concat(all_data, ignore_index=True)


In [ ]:
shows['if_qualified'] = shows['Participant'].apply(lambda s: 1 if 'qualified' in s else 0)
shows['Participant'] = shows['Participant'].str.replace('qualified','')
shows['Half'] = shows['Half'].str.replace(r'[^0-9]','', regex=True)
shows['Rank'] = shows['Rank'].str.replace(r'[^0-9]','', regex=True)
shows = shows.rename(columns = {'link0':'Country_link', 'link1':'Participant_link','R/O\n\nSort descending\n\n\n':'R/O'})
shows['Country_link'] = 'https://eurovision.tv' + shows['Country_link']
shows['Participant_link'] = 'https://eurovision.tv' + shows['Participant_link']
shows = shows[['event', 'show','R/O','Half','Country','Participant','Song','Rank','Points','Country_link','Participant_link']]
shows.head()


,event,show,R/O,Half,Country,Participant,Song,Rank,Points,Country_link,Participant_link
0,turin-2022,first-semi-final,1,1,Albania,Ronela Hajati,Sekret,12,58,https://eurovision.tv/country/albania,https://eurovision.tv/participant/ronela-hajat...
1,turin-2022,first-semi-final,2,1,Latvia,Citi Zēni,Eat Your Salad,14,55,https://eurovision.tv/country/latvia,https://eurovision.tv/participant/citi-zeni-22
2,turin-2022,first-semi-final,3,1,Lithuania,Monika Liu,Sentimentai,7,159,https://eurovision.tv/country/lithuania,https://eurovision.tv/participant/monika-liu-22
3,turin-2022,first-semi-final,4,1,Switzerland,Marius Bear,Boys Do Cry,9,118,https://eurovision.tv/country/switzerland,https://eurovision.tv/participant/marius-bear-22
4,turin-2022,first-semi-final,5,1,Slovenia,LPS,Disko,17,15,https://eurovision.tv/country/slovenia,https://eurovision.tv/participant/lps-22


In [ ]:
add_data(shows, 'shows')

#Collecting votes for every year



I fixed some links with my hands - not cool

In [7]:
from gspread_dataframe import get_as_dataframe
worksheet = gc.open(default_spreadsheet_title).worksheet('shows')
shows_links = get_as_dataframe(worksheet)[['event','show','Results_link']]
shows_links.head()

,event,show,Results_link
0,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...
1,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...
2,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...
3,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...
4,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...


In [16]:
def get_voting_table(soup):
  voted_for = soup.select_one('.details-tab__voted-for-participant')
  participant_name = voted_for.select_one('.participant-name').text.strip()
  voting_tables = voted_for.select('.views-element-container')

  data = []
  for t in voting_tables:
    if t.div:
      table_title = t.div.div.text.strip()
      voting_type = table_title.split()[-1] if 'from' in table_title else 'other'

      for table_row in t.select('.details-tab__voted-row'):
        points = table_row.select_one('div.details-tab__voted-point').text.strip()
        countries = table_row.select('div.details-tab__voter')
        row = []
        for c in countries:
         row.append([voting_type, participant_name, c.text.strip(), points])
        data.extend(row)
  if not data:
    data = [['all_votings', participant_name, 'every_country', 0]]
  columns = ['voting_type', 'points_for', 'points_from', 'points']
  return pd.DataFrame(data=data, columns = columns)
  
    

In [ ]:
all_data = []
failed_urls = []
for i, row in shows_links.iterrows():
  r = requests.get(url = row['Results_link'],headers={'User-Agent': 'Mozilla/5.0'})
  if r.ok:
    try:
      soup = BeautifulSoup(r.text,'html.parser')
      df = get_voting_table(soup)
      df['event'] = row['event']
      df['show'] = row['show']
      all_data.append(df)
      print(row['Results_link'])
    except:
      failed_urls.append(row['Results_link'])
  else:
    print(r.staus_code)
  sleep(random.randrange(3,5,1))
votes = pd.concat(all_data, ignore_index=True)
votes.head(30)


In [23]:
votes.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/votes.csv')

In [27]:
votes.head()

,voting_type,points_for,points_from,points,event,show
0,televoters,Albania,Greece,12,turin-2022,first-semi-final
1,televoters,Albania,Italy,8,turin-2022,first-semi-final
2,televoters,Albania,Switzerland,8,turin-2022,first-semi-final
3,televoters,Albania,Slovenia,6,turin-2022,first-semi-final
4,televoters,Albania,France,5,turin-2022,first-semi-final


In [28]:
votes.tail()

,voting_type,points_for,points_from,points,event,show
20482,other,Austria,Iceland,3,stockholm-2000,final
20483,other,Austria,Malta,2,stockholm-2000,final
20484,other,Austria,Ireland,2,stockholm-2000,final
20485,other,Austria,Germany,2,stockholm-2000,final
20486,other,Austria,United Kingdom,1,stockholm-2000,final


In [29]:
failed_urls

['https://eurovision.tv/event/helsinki-2007/grand-final/results/fyr-macedonia',
 'https://eurovision.tv/event/athens-2006/grand-final/results/fyr-macedonia',
 'https://eurovision.tv/event/kyiv-2005/semi-final/results/fyr-macedonia']

In [ ]:
failed_urls = ['https://eurovision.tv/event/helsinki-2007/grand-final/results/fyr-macedonia',
 'https://eurovision.tv/event/athens-2006/grand-final/results/fyr-macedonia',
 'https://eurovision.tv/event/kyiv-2005/semi-final/results/fyr-macedonia']